In [1]:
import pandas as pd
import numpy as np
import joblib

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import load_model
from pathlib import Path
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from sklearn.decomposition import PCA
import pickle
import math
import gc
from collections import defaultdict
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Lambda
from tensorflow.keras import backend as K
from tensorflow.keras.metrics import AUC

In [57]:
##### FUNCTIONS #####


def pairwise_combinations_with_ids_and_values(X):
    n_samples = X.shape[0]
    generated_pairs = set()  # To store generated pairs and avoid duplicates
    X_pairs = []
    X_pair_ids = []

    for i in range(n_samples):
        date_i = X[i, 0]
        same_date_indices = [j for j in range(i + 1, n_samples) if X[j, 0] == date_i]
        for j in same_date_indices:
            pair_key = tuple(sorted((i, j)))  # Create a unique key for the pair
            if pair_key not in generated_pairs:  # Check if pair is already generated
                X_pair = np.array([X[i, 2:], X[j, 2:]])
                X_pair_id = np.array([X[i, :2], X[j, :2]])
                generated_pairs.add(pair_key)
                X_pairs.append(X_pair)
                X_pair_ids.append(X_pair_id)

    return np.array(X_pairs, dtype='float32'), np.array(X_pair_ids)




def pairwise_generator5(X, y, batch_size):
    n_samples = X.shape[0]

    # Create a dictionary to store indices for each date
    date_indices = defaultdict(list)
    for i in range(n_samples):
        date_indices[X[i, 0]].append(i)

    while True:
        X_batch = []
        y_batch = []

        for date_i in date_indices.keys():
            same_date_indices = date_indices[date_i]

            if len(same_date_indices) < 2:
                continue

            # Create pairs for all samples of the same date
            for i in range(len(same_date_indices)):
                for j in range(i+1, len(same_date_indices)):
                    before_comparison = y[same_date_indices]
                    # Ensure the first sample in each pair is the one with the higher target value
                    if y[same_date_indices[i]][2] > y[same_date_indices[j]][2]:
                        X_pair = [X[same_date_indices[i], 2:], X[same_date_indices[j], 2:]]  # Ignore the first two columns
                        y_pair = 1
                    else:
                        X_pair = [X[same_date_indices[j], 2:], X[same_date_indices[i], 2:]]  # Ignore the first two columns
                        y_pair = 0

                    X_batch.append(X_pair)
                    y_batch.append(y_pair)

                    if len(X_batch) == batch_size:
                        X_batch_array = np.array(X_batch, dtype='float32')
                        yield [X_batch_array[:, 0], X_batch_array[:, 1]], np.array(y_batch, dtype='float32').reshape(-1,1)
                        X_batch = []
                        y_batch = []  # Reset batches

        if X_batch:  # If there are any remaining samples that didn't make a full batch
            X_batch_array = np.array(X_batch, dtype='float32')
            yield [X_batch_array[:, 0], X_batch_array[:, 1]], np.array(y_batch, dtype='float32').reshape(-1,1)


def create_siamese_model(input_shape):
    # Define the tensors for the two input images
    left_input = Input(input_shape)
    right_input = Input(input_shape)

    # Neural Network
    model = Sequential()
    model.add(Dense(400, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(20, activation='relu'))

    # Generate the encodings for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)

    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])

    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1, activation='sigmoid')(L1_distance)

    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input, right_input], outputs=prediction)

    # Return the model
    return siamese_net

In [58]:
##### TRAIN #####
def train(X_train: pd.DataFrame, y_train: pd.DataFrame, model_directory_path: str = "../resources") -> None:
    max_date = X_train['date'].max()
    min_date = 0
    X_train_orig = X_train[(X_train['date'] < max_date) & (X_train['date'] > min_date)]
    X_test = X_train[X_train['date'] == max_date]
    y_train_orig = y_train[(y_train['date'] < max_date) & (y_train['date'] > min_date)]
    y_test = y_train[y_train['date'] == max_date]

    #Scaling
    X_ids = np.asarray(X_train_orig[['date', 'id']])
    X_scale_pca = X_train_orig.drop(columns=['date', 'id'])
    X_scale_pca = np.asarray(X_scale_pca)


    X_test_ids = np.asarray(X_test[['date', 'id']])
    X_test_scale_pca = X_test.drop(columns=['date', 'id'])
    X_test_scale_pca = np.asarray(X_test_scale_pca)

    #PCA
    n_components = 120
    pca = PCA(n_components=n_components)
    pca_features = pca.fit_transform(X_scale_pca)
    X_train_concat = np.concatenate((X_ids, pca_features), axis=1)
    y_train = np.asarray(y_train)

    pca_features_test = pca.transform(X_test_scale_pca)
    X_test_concat = np.concatenate((X_test_ids, pca_features_test), axis=1)
    y_test = np.asarray(y_test)


    #Save out Scaler and PCA
    with open(Path(model_directory_path) / 'pca.pkl', 'wb') as file:
        pickle.dump(pca, file)

    date_list = list(set(X_train_orig['date']))

    batch_size = 3500
    train_generator = pairwise_generator5(X_train_concat, y_train, batch_size)
    test_generator= pairwise_generator5(X_test_concat, y_test, batch_size)

    print(X_train_concat.shape)


    #Model Training
    model_pathname = Path('../resources') / "model.keras"

    if model_pathname.is_file():
        early_stopping = EarlyStopping(
            monitor='val_loss',
            patience=8,
            verbose=0,
            mode='auto',
            baseline=None)

        history = model.fit(
            train_generator,
            batch_size=batch_size,
            steps_per_epoch = 1000,
            epochs=1000,
            validation_data=test_generator,
            validation_steps = 500,
            callbacks=[mc, early_stopping],
            shuffle=True,
            use_multiprocessing=False,
            verbose=1
        )

    else:
        #Neural Network Model
        mc = ModelCheckpoint(model_pathname, monitor='val_loss', mode='min', verbose=1, save_best_only=True)

        early_stopping = EarlyStopping(
            monitor='val_loss',
            patience=8,
            verbose=1,
            mode='auto',
            baseline=None)

        model = create_siamese_model((X_train_concat.shape[1] - 2))

        optimizer = keras.optimizers.Adam(learning_rate=0.001)

        model.compile(optimizer=optimizer,
                      loss='binary_crossentropy',
                      metrics=[AUC(name='auc')])

        history = model.fit(
            train_generator,
            batch_size=batch_size,
            steps_per_epoch=1000,
            epochs=1000,
            validation_data=test_generator,
            validation_steps=500,
            callbacks=[mc, early_stopping],
            shuffle=True,
            use_multiprocessing=False,
            verbose=1
        )

        model.save(model_pathname)




    gc.collect()

    print("Finished All Training")

In [59]:
def infer(X_test: pd.DataFrame, model_directory_path: str = "../resources") -> pd.DataFrame:
    X_test_orig = X_test.copy()

    # Load Scaler
    with open(Path(model_directory_path) / 'scaler.pkl', 'rb') as file:
        scaler = pickle.load(file)


    # Scaling
    X_ids = np.asarray(X_test_orig[['date', 'id']])
    X_scale_pca = X_test_orig.drop(columns=['date', 'id'])
    X_scale_pca = scaler.transform(X_scale_pca)

    X_test_concat = np.concatenate((X_ids, X_scale_pca), axis=1)


    result_df = pd.DataFrame(columns=['date', 'id', 'value'])

    # Load Model
    model_pathname = Path(model_directory_path) / "model.keras"
    model = load_model(model_pathname)

    # Pairwise Transformation using the pairwise generator
    batch_size = 3500
    X_test, X_test_ids = pairwise_combinations_with_ids_and_values(X_test_concat)

    print("Predicting for Test Data")
    preds = model.predict(X_test)

    preds_df = pd.DataFrame({'id': X_test_ids[:, 0, 1].flatten(), 'date': X_test_ids[:, 0, 0].flatten(), 'value': preds.flatten()})
    preds_df = preds_df.groupby(['date', 'id']).mean().reset_index()

    result_df = pd.merge(X_test_orig, preds_df, on=['id', 'date'], how='left')
    result_df['value'] = result_df['value'].fillna(result_df['value'].mean())

    minmax = MinMaxScaler(feature_range=(-1, 1))
    result_df['value'] = minmax.fit_transform(result_df[['value']])

    print("Finished predicting Test Data")

    return result_df

In [60]:
X_train = pd.read_parquet('../data/X_train.parquet')
y_train = pd.read_parquet('../data/y_train.parquet')
X_test = pd.read_parquet('../data/X_test.parquet')

In [61]:
train(X_train, y_train)

(737864, 122)
Epoch 1/1000
1000/1000 [==============================] - ETA: 0s - loss: 0.6961 - auc: 0.5061
Epoch 1: val_loss improved from inf to 0.69313, saving model to ..\resources\model.keras
1000/1000 [==============================] - 543s 543ms/step - loss: 0.6961 - auc: 0.5061 - val_loss: 0.6931 - val_auc: 0.4985
Epoch 2/1000
1000/1000 [==============================] - ETA: 0s - loss: 0.6939 - auc: 0.5002
Epoch 2: val_loss did not improve from 0.69313
1000/1000 [==============================] - 565s 566ms/step - loss: 0.6939 - auc: 0.5002 - val_loss: 0.6933 - val_auc: 0.4987
Epoch 3/1000
1000/1000 [==============================] - ETA: 0s - loss: 0.6944 - auc: 0.4980
Epoch 3: val_loss did not improve from 0.69313
1000/1000 [==============================] - 576s 576ms/step - loss: 0.6944 - auc: 0.4980 - val_loss: 0.6943 - val_auc: 0.5040
Epoch 4/1000
1000/1000 [==============================] - ETA: 0s - loss: 0.6936 - auc: 0.5019
Epoch 4: val_loss did not improve from 0.6

KeyboardInterrupt: 

In [ ]:
results = infer(X_test)

In [5]:
X_train = pd.read_parquet('../data/X_train.parquet')
y_train = pd.read_parquet('../data/y_train.parquet')
X_test = pd.read_parquet('../data/X_test.parquet')

model_directory_path = '../resources'
max_date = X_train['date'].max()
min_date = 0
X_train_orig = X_train[(X_train['date'] < max_date) & (X_train['date'] > min_date)]
X_test = X_train[X_train['date'] == max_date]
y_train_orig = y_train[(y_train['date'] < max_date) & (y_train['date'] > min_date)]
y_test = y_train[y_train['date'] == max_date]

#Scaling
scaler = StandardScaler()
X_ids = np.asarray(X_train_orig[['date', 'id']])
X_scale_pca = X_train_orig.drop(columns=['date', 'id'])
X_scale_pca = scaler.fit_transform(X_scale_pca)


X_test_ids = np.asarray(X_test[['date', 'id']])
X_test_scale_pca = X_test.drop(columns=['date', 'id'])
X_test_scale_pca = scaler.transform(X_test_scale_pca)

#PCA
n_components = 120
pca = PCA(n_components=n_components)
pca_features = pca.fit_transform(X_scale_pca)
X_train_concat = np.concatenate((X_ids, pca_features), axis=1)
y_train = np.asarray(y_train)

pca_features_test = pca.transform(X_test_scale_pca)
X_test_concat = np.concatenate((X_test_ids, pca_features_test), axis=1)
y_test = np.asarray(y_test)


#Save out Scaler and PCA
with open(Path(model_directory_path) / 'scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

with open(Path(model_directory_path) / 'pca.pkl', 'wb') as file:
    pickle.dump(pca, file)

date_list = list(set(X_train_orig['date']))

batch_size = 500
train_generator = pairwise_generator5(X_train_concat, y_train, batch_size)
test_generator= pairwise_generator5(X_train_concat, y_train, batch_size)

In [26]:
x, y = next(train_generator)

In [41]:
x[0][0].shape

(120,)

In [18]:
test[:,0]

TypeError: tuple indices must be integers or slices, not tuple